In [1]:
import numpy as np
import math
import scipy.stats as stats
from scipy.linalg import sqrtm
import scipy
import matplotlib.pyplot as plt
import time
import datetime
import random
import statistics

import numba
from numba import jit, njit
from numba.typed import List

import independent_functions as i_f

In [2]:
def fdep_marginals_pooled_ruth(alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR):

    block_beta_agg = []
    
    #  columns order: marglocfdrFDR, marglocfdrpFDR, marglocfdrmFDR
    minprob_mat = np.zeros((1,6),float)
    lev_mat = np.zeros((1,6),float) 
    pow_mat = np.zeros((1,6),float) 
    ev_mat = np.zeros((1,6),float) 
    er_mat = np.zeros((1,6),float) 
    
    
    for i in range(len(mu0)):
        # - - - BLOCK BETA - - - creating vector Z from each group, but straight afterwards - concating it
        block_beta = i_f.my_rbeta(num_hypo[i], 
                                prob_to_1[i], 
                                mu0[i], 
                                mu1[i], 
                                variance_0[i], 
                                variance_1[i],
                                False)
        block_beta_agg.append(block_beta)
        
    # concating the Z vectors to a one coherent vector
    block_beta_agg_stacked = np.hstack(block_beta_agg)   
    
    # - - - creating locfdr for the pooled rule - - -
    
    ### NUMERATOR
    first_group_prob = (num_hypo[0])/(num_hypo[0]+num_hypo[1])
    second_group_prob = (num_hypo[1])/(num_hypo[0]+num_hypo[1])
    Ph0 = (1-prob_to_1[0])*(first_group_prob) + (1-prob_to_1[1])*(second_group_prob)
    # Assuming that both null distributions in the 2 groups are the same (mu0 & variance0 has the same values in their 2 indexes of groups), other wise, needed to change to the same logic of partition as Ph0
    dist_0 = stats.norm(mu0[0], np.sqrt(variance_0[0]))
    Pzh0 = dist_0.pdf(block_beta_agg_stacked)

    NUMERATOR = Pzh0*Ph0


    ### DENOMINATOR
    Ph1c1 = prob_to_1[0]*first_group_prob
    Ph1c2 = prob_to_1[1]*second_group_prob

    dist_1_1 = stats.norm(mu1[0], np.sqrt(variance_1[0]))
    Pzh1c1 = dist_1_1.pdf(block_beta_agg_stacked)
    dist_1_2 = stats.norm(mu1[1], np.sqrt(variance_1[1]))
    Pzh1c2 = dist_1_2.pdf(block_beta_agg_stacked)

    DENOMINATOR = NUMERATOR + Pzh1c1*Ph1c1 + Pzh1c2*Ph1c2


    ### marglocfdr
    marglocfdr = NUMERATOR / DENOMINATOR
    
    
    # - - - creating a marginal & b marginal - - - 
    omarglocfdr = np.sort(marglocfdr)
    amarg = 1 - omarglocfdr
    omarglocfdr_numba = numba.typed.List(omarglocfdr)    
    bmarg = np.array(i_f.BZCpp_numba_jit(omarglocfdr_numba)) # it WAS np.array but now it's a LIST of np.arrays so that when you take an index from it, it will be fine
    
    # marglocfdrFDR 
    lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = i_f.FDR_Generic_structure (mus=musMargFDR, a=amarg, b_1=bmarg, 
                                                                               b_2=bmarg, ind=3, 
                                                                               lev_mat=lev_mat, 
                                                                               pow_mat=pow_mat, 
                                                                               minprob_mat=minprob_mat, 
                                                                               ev_mat=ev_mat, 
                                                                               er_mat=er_mat,  
                                                                               olocfdr_function=omarglocfdr, 
                                                                               alpha = alpha)

    return lev_mat, pow_mat, minprob_mat, ev_mat, er_mat


In [3]:
#musMargFDR_scalar = [1425.5]   # between 1432.3160887391211 to 1419.
alpha = 0.1

# Cai & sun 09' setting - Study 1# --- 1
# All of the relevant constants, BUT the one that varies

num_hypo = [300, 150] # before optimization: [3000, 1500]

mu0 = [0, 0]
mu1 = [-2, 4]

variance_0 = [1, 1]
variance_1 = [1, 1]

In [4]:
# prob_to_1 = [0.03, 0.1]
# prob_to_1 = [0.15, 0.1]
# prob_to_1 = [0.27, 0.1]

# finding unit root

In [5]:
def func_marg_pooled_FDR(mu_FDR_variable, iter_num, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1):
    start = time.time()
    lev_mat_agg = 0
    pow_mat_agg = 0
    minprob_mat_agg = 0
    ev_mat_agg = 0
    er_mat_agg = 0
    
    for i in range(iter_num):
        if i == 1000:
            print("this is iteration no. 1000")
        if i == 4000:
            print("this is iteration no. 4000")
            
            
        lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = fdep_marginals_pooled_ruth(alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, 
                                                                                [mu_FDR_variable])
        lev_mat_agg += lev_mat[0]
        pow_mat_agg += pow_mat[0]
        minprob_mat_agg += minprob_mat[0]
        ev_mat_agg += ev_mat[0]
        er_mat_agg += er_mat[0]
        
    # R & S RETURN
    pow_mat_r = pow_mat_agg / iter_num
    lev_mat_r = lev_mat_agg / iter_num
    ev_mat_r = ev_mat_agg / iter_num
    er_mat_r = er_mat_agg / iter_num
    minprob_mat_r = minprob_mat_agg / iter_num

    print("This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: ")
    print(mu_FDR_variable)
    stop = time.time()
    duration = stop-start
    print(f"Time this {iter_num} iteration run took: " + str(duration))
    print("This is the calculation right now, aiming for 0: ")
    print(lev_mat_r[3] - alpha)
    
    
    return(lev_mat_r[3] - alpha)

In [28]:
%%time
iter_num = 5000 

prob_to_1 = [0.03, 0.1]

root_pooled_FDR = scipy.optimize.brentq(f=func_marg_pooled_FDR, a=17, b=25, args=(iter_num, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1))#, maxiter=17, disp=False) 
root_pooled_FDR

this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
17.0
Time this 5000 iteration run took: 77.39853811264038
This is the calculation right now, aiming for 0: 
0.04276275964758086
this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
25.0
Time this 5000 iteration run took: 82.20987248420715
This is the calculation right now, aiming for 0: 
-0.018242772878612493
this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
22.60772216903052
Time this 5000 iteration run took: 78.29210209846497
This is the calculation right now, aiming for 0: 
-0.005944278592142291
this is iteration no. 1000


KeyboardInterrupt: 

In [ ]:
%%time
iter_num = 5000 

prob_to_1 = [0.15, 0.1]

root_pooled_FDR = scipy.optimize.brentq(f=func_marg_pooled_FDR, a=85, b=115, args=(iter_num, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1))#, maxiter=17, disp=False) 
root_pooled_FDR

this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
85.0
Time this 5000 iteration run took: 76.82020592689514
This is the calculation right now, aiming for 0: 
0.035590381129453036
this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
115.0
Time this 5000 iteration run took: 77.31794881820679
This is the calculation right now, aiming for 0: 
-0.0220129723766741
this is iteration no. 1000


In [ ]:
%%time
iter_num = 5000 

prob_to_1 = [0.27, 0.1]

root_pooled_FDR = scipy.optimize.brentq(f=func_marg_pooled_FDR, a=190, b=225, args=(iter_num, alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1))#, maxiter=17, disp=False) 
root_pooled_FDR

this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
190.0
Time this 5000 iteration run took: 82.9924488067627
This is the calculation right now, aiming for 0: 
0.015358289019904325
this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
225.0
Time this 5000 iteration run took: 82.90010499954224
This is the calculation right now, aiming for 0: 
-0.01119981676795724
this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
210.2401526671505
Time this 5000 iteration run took: 82.46109986305237
This is the calculation right now, aiming for 0: 
-0.001534851692168021
this is iteration no. 1000
this is iteration no. 4000
This is MU now!@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@: 
208.10916748292573
Tim

# - - - backup: the new locfdr creation

In [ ]:
### NUMERATOR

first_group_prob = (num_hypo[0])/(num_hypo[0]+num_hypo[1])
second_group_prob = (num_hypo[1])/(num_hypo[0]+num_hypo[1])
Ph0 = (1-prob_to_1[0])*(first_group_prob) + (1-prob_to_1[1])*(second_group_prob)
# Assuming that both null distributions in the 2 groups are the same (mu0 & variance0 has the same values in their 2 indexes of groups), other wise, needed to change to the same logic of partition as Ph0
dist_0 = stats.norm(mu0[0], np.sqrt(variance_0[0]))
Pzh0 = dist_0.pdf(block_beta_agg_stacked)

NUMERATOR = Pzh0*Ph0


### DENOMINATOR

Ph1c1 = prob_to_1[0]*first_group_prob
Ph1c2 = prob_to_1[1]*second_group_prob

dist_1_1 = stats.norm(mu1[0], np.sqrt(variance_1[0]))
Pzh1c1 = dist_1_1.pdf(block_beta_agg_stacked)
dist_1_2 = stats.norm(mu1[1], np.sqrt(variance_1[1]))
Pzh1c2 = dist_1_2.pdf(block_beta_agg_stacked)

DENOMINATOR = NUMERATOR + Pzh1c1*Ph1c1 + Pzh1c2*Ph1c2


### marglocfdr
marglocfdr = NUMERATOR / DENOMINATOR

In [19]:

lev_mat, pow_mat, minprob_mat, ev_mat, er_mat = fdep_marginals_pooled_ruth(alpha, num_hypo, prob_to_1, mu0, mu1, variance_0, variance_1, musMargFDR_scalar)
# lev_mat, pow_mat, minprob_mat, ev_mat, er_mat 